<a href="https://colab.research.google.com/github/SojiroNishimura/deeplearning-from-scratch/blob/master/chapter5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 誤差逆伝播法
ニューラルネットワークの学習を行うためには各層での勾配を求める必要がある。勾配は損失関数を数値微分することで求められるが、数値微分の計算は時間がかかる。誤差逆伝播法を使うことで各時点における出力を解析的に微分することができるため、効率的（=高速）に学習が完了する。

## 連鎖律
ある計算をグラフで表現した場合、左(始点)から右(終点)に向けて計算結果を伝達させていくことを**順伝播**という。逆に右(終点)から左(始点)に向けて、入力と各ノードにおける局所的な微分の値を掛け合わせていくことを**逆伝播**という。

$y=f(x)$という計算の場合、逆伝播の入力を$E$とすると$E$とノードの計算を微分したもの=$\frac{\partial y}{\partial x}$を掛け合わせた$E\frac{\partial y}{\partial x}$がそのノードの逆伝播の出力となり、次の(1つ前の）ノードへの入力となる。

### 合成関数
合成関数=複数の関数によって構成される関数。例として$z=(x+y)^2$は以下の2式で構成される。

$z = t^2$\
$t = x + y$

連鎖律は以下のように定義される。

> ある関数が合成関数で表される場合、その合成関数の微分は**合成関数を構成するそれぞれの関数の微分の積**によって表すことができる。

上記の式に当てはめると以下のようになる。

$$\frac{\partial z}{\partial x} = \frac{\partial z}{\partial t}\frac{\partial t}{\partial x} = \frac{\partial z}{\partial x}$$

先ほどの式に適用すると以下のとおり。

$$\frac{\partial z}{\partial t} = 2t$$\
$$\frac{\partial t}{\partial x} = 1$$\
$$\frac{\partial z}{\partial x} = \frac{\partial z}{\partial t}\frac{\partial t}{\partial x} = 2t\cdot1 = 2(x + y)$$

## 逆伝播
「加算」「乗算」の各計算の逆伝播。

### 加算ノードの逆伝播
$z = x + y$の場合

$$\frac{\partial z}{\partial x} = 1$$\
$$\frac{\partial z}{\partial y} = 1$$

右側(上流)からの入力が$\frac{\partial L}{\partial z}$とすると、加算ノードにおける逆伝播の出力はそれぞれ$\frac{\partial L}{\partial z}\cdot 1$となる。

### 乗算ノードの逆伝播
$z = xy$の場合

$$\frac{\partial z}{\partial x} = y$$\
$$\frac{\partial z}{\partial y} = x$$

右側(上流)からの入力が$\frac{\partial L}{\partial z}$とすると、逆伝播の出力はそれぞれ以下のようになる。

x方向への逆伝播：$\frac{\partial L}{\partial z}\cdot y$\
y方向への逆伝播：$\frac{\partial L}{\partial z}\cdot x$

各ノードは以下のように実装できる。

In [0]:
# 乗算レイヤ
class MulLayer:
  def __init__(self):
    self.x = None
    self.y = None
    
  def forward(self, x, y):
    self.x = x
    self.y = y
    out = x * y
    
    return out
  
  def backward(self, dout):
    dx = dout * self.y # xとyをひっくり返す
    dy = dout * self.x
    
    return dx, dy

In [2]:
# 消費税１０%で1個110円のりんごを2個買う場合の順伝播での計算
apple = 100
apple_num = 2
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [3]:
# 各変数の微分(backward)
dprice = 1

# 順伝播の時とは逆向きに計算を行う
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


In [0]:
# 加算レイヤ
class AddLayer:
  def __init__(self):
    pass
  
  def forward(self, x, y):
    out = x + y
    return out
  
  def backward(self, dout):
    dx = dout * 1
    dy = dout * 1
    return dx, dy

In [7]:
# 消費税10%で100円のりんご2個と150円のみかん3個を買う場合の順伝播での計算
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num) # (1)
orange_price = mul_orange_layer.forward(orange, orange_num) # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price) # (3)
price = mul_tax_layer.forward(all_price, tax) # (4)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice) # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price) # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price) # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price) # (1)

print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650
